# BINF TP3 - Algorithmes d'alignement par paire

Dans ce TP nous allons manipuler les algorithmes d'alignement par paire.

# Exercice 0 - Echauffement

Q1. Donnez le score de la superposition :

|       |       |
| :---: | :---: |
x       | ATGTCATGA---TAC |
y       | AT--CTAAATGTTAC |


étant donne le schéma d'évaluation :

|       | A     | T     | G     | C     |
| :---: | :---: | :---: | :---: | :---: |
| **A** | 1     | -1    | -1    | -1    |
| **T** | -1    | 1     | -1    | -1    |
| **G** | -1    | -1    | 1     | -1    |
| **C** | -1    | -1    | -1    | 1     |

et

$\gamma(g) = 0.5 |g| + 0.5$

```markdown
1+1+1-1-1-1+1+1+1+1 = 4
2 gap = 1.5 + 2 = 3.5
result = 4-3.5 = 0.5
```

Q2. Alignez les séquences suivantes avec l'algorithme de Levenshtein :  x = ATG et y = ACTG.

```markdown
ATG | ACTG -> 0

TG | CTG -> 1 + min (G|TG, TG|TG, G|CTG) -> TG|TG = 0

result = A_TG|ACTG score = 1
```

Q3.	Alignez les séquences suivantes avec l'algorithme de Needleman-Wunsch global x = TAT et y = ATGAC en considérant le schéma d'évaluation suivant

|       | A       | T       | G       | C     |
| :---: | :---:   | :---:   | :---:   | :---: |
| **A** | 1       | -0.5    | -0.5    | -0.5  |
| **T** | -0.5    | 1       | -0.5    | -0.5  |
| **G** | -0.5    | -0.5    | 1       | -0.5  |
| **C** | -0.5    | -0.5    | -0.5    | 1     |

et

$\gamma(g) = 0.5 |g|$

ATGAC
_T_AT     

S: 
|       | V       | A       | T       | G       | A     | C     |
| :---: | :---:   | :---:   | :---:   | :---:   | :---: | :---: |
| **V** | 0       | -0.5    | -1      | -1.5    | -2    | -2.5  |
| **T** | -0.5    | -0.5    | 0.5     | 0       | -0.5  | -1    |
| **A** | -1      | 0.5     | 0       | 0       | 1     | 0.5   |
| **T** | -1.5    | 0       | 1.5     | 1       | 0.5   | 0.5   |

B:


|       | V       | A       | T       | G       | A    | C     |
| :---: | :---:   | :---:   | :---:   | :---:   | :---: | :---: |
| **V** | F       | G       | G       | G       | G     | G     |
| **T** | M       | DIA     | DIA     | G       | G     | G     |
| **A** | M       | DIA     | G       | DIA     | DIA   | G     |
| **T** | M       | M       | DIA     | G       | G     | DIA   |



Q4. Alignez les séquences suivantes avec l'algorithme de Smith-Waterman x = TTGG y = ATGAC en utilisant le schéma d'évaluation de la question précédente.


```markdown
Votre réponse ici
```

# Exercice 1 : Algorithme de Levenshtein - version récursive

Q1. Ecrivez une fonction

levenshtein(x: str, y: str) -> int

qui retourne la distance de Levenshtein entre les séquences x et y en utilisant la  version récursive de l'algorithme.

In [8]:
def levenshtein(x,y):
    if len(x) == 0:
        return len(y)
    elif len(y) == 0:
        return len(x)
    elif x[0] == y[0]:
        return levenshtein(x[1:],y[1:])
    else:
        return 1 + min(levenshtein(x[1:],y[1:]),levenshtein(x[1:],y),levenshtein(x,y[1:]))

Q2. Vous pouvez tester votre code sur les exemples suivants:


*   $L('CCAG', 'CA') = 2$
*   $L('CCGT', 'CGTCA') = 3$
*   $L(AY678264^*, OQ870305^*) = 310$

$^*$ ids genbank de deux sequences.

In [13]:
print(levenshtein("CCAG", "CA"))
print(levenshtein("CCGT", "CGTCA"))
print(levenshtein("AY678264", "OQ870305"))

2
3
7


# Exercice 2 : Algorithme de Smith-Waterman - version itérative

Q1. Ecrivez la fonction

sw_fwd(x: str, y: str, cmap: dict, sigma: array, (go, ge): list) -> (array, array)

qui construit les matrices $S$ et $B$ en utilisant l'algorithme de Smith-Waterman pour aligner les séquences x et y suivant le schéma d'évaluation donné par la matrice de substitution $\Sigma$ et la fonction d'évaluation des trous $\gamma(n)= g_o + g_e \times n$. Le dictionnaire cmap donne la position des différents nucléotides dans la matrice $\Sigma$. La fonction retourne la paire de matrices de score $S$ et de retour $B$.

In [45]:
def sw_fwd(x,y,cmap,sigma,go,ge):
    S = [[0 for _ in range(len(y) + 1)] for _ in range(len(x) + 1)]
    B = [[(0,0) for _ in range(len(y) + 1)] for _ in range(len(x) + 1)]
    
    for i in range(1,len(x) + 1):
        for j in range(1,len(y) + 1):
            a = sigma[cmap[x[i-1]]][cmap[y[j-1]]] + S[i-1][j-1]
            b = S[i][j-1] - (go + ge)
            c = S[i-1][j] - (go + ge)
            n = max(a,b,c,0)
            S[i][j] = n

            if n == 0:
                B[i][j] = (0,0)
            elif n == a:
                B[i][j] = (-1,-1)
            elif n == b:
                B[i][j] = (0,-1)
            elif n == c:
                B[i][j] = (-1,0)
            else:
                B[i][j] = (0,0)
    return S,B

Q2. Ecrivez la fonction

sw_bwd(x: str, y: str, S: array, B: array) -> (str, str, float)

qui effectue l'etape de retour de l'algorithme de Smith-Waterman etant donné les séquences $x$ et $y$ et les matrices de score $S$ et de retour $B$. La fonction retourne un tuple contenant les alignements des séquences x et y et le score de l'alignement.

In [49]:
def find_max(matrix):
    max_value = float('-inf') 
    max_indices = (-1, -1) 

    for i in range(len(matrix)):
        for j in range(len(matrix[i])): 
            if matrix[i][j] > max_value:
                max_value = matrix[i][j]
                max_indices = (i, j) 

    return max_value, max_indices

def sw_bwd(x,y,S,B):
    score, (i,j) = find_max(S)
    result_X = ""
    result_Y = ""
    Ni,Nj = B[i][j]
    while (Ni,Nj) != (0,0):
        if (Ni == -1):
            result_X = x[i-1] + result_X
            i = i-1
        else:
            result_X = "_" + result_X
        if (Nj == -1):
            result_Y = y[j-1] + result_Y
            j = j-1
        else:
            result_Y = "_" + result_Y
        Ni,Nj = B[i][j]
    return (result_X, result_Y, score)

Q3. Vous pouvez tester votre code en utilisant le schéma d'évaluation suivant :

In [52]:
cmap = {"A": 0, "T": 1, "G": 2, "C": 3}
m = [[1, -0.5, -0.5, -0.5],
      [-0.5, 1, -0.5, -0.5],
      [-0.5, -0.5, 1, -0.5],
      [-0.5, -0.5, -0.5, 1]]
go = 0
ge = 0.5

S,B = sw_fwd("TCGC", "CTTAG", cmap, m, go, ge)

print(sw_bwd("TCGC", "CTTAG", S, B))

('TCG', 'TAG', 1.5)


*   $SW('TCGC', 'CTTAG')$ retourne un score de $1.5$ à la position $(3,5)$ et l'alignement

In [53]:
HTML("<table align='left' style='font-family:Courier New'><tr><th>x:</th><th>TCG</th></tr><tr><th>y:</th><th>TAG</th></tr></table>")

NameError: name 'HTML' is not defined

*   $SW(AY678264^*, OQ870305^*)$ retourne un score de $342.1$ à la position $(708,717)$ et l'alignement

In [54]:
from IPython.display import HTML
HTML("<table width='300px' align='left' style='font-family:Courier New'><tr><th>x:</th><th nowrap='nowrap'>ATGGTGAGCAAGGGCGAGGAGGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGC-A-CATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAG---GGCGAGGGCGAGGGC--CGCC-CCTACGAGGGCACCCAGACCGC-CAAGCTGAAGGTG-ACCA-AGG---G-TGGCC---CCCT-GCCCTTCGCCT-GGGA-CATCCTGTCC--C--C-T-CAGTTCATGT-A-CGGCT-CCAAGGCCTACGTG-A--AGCAC--C--C--C--G-CCGACATCCCCG-A--CTAC-T--TGAAGCTG-TCCTTC--C--C-----CGA-GG--GCTTCAAGTGGGAGCG-CGTGATGAACTTCGAGGACGGCGGCGTGGTG-ACCG--T-GA-C-CCAGGAC-TC--CTCCCTGCAGGACGGCGAGTTCATCTACAAGGTG---AAGCTGCGCGGCACCAACTTCCCCT-CCGACGGCCCCGTA-ATGCA-GAAGAAGACCATGGGCTG--GGA-GGCCTCCTCCGAGCGGATGTACCCCGAGGA-CGGCGCC-CTGAAGGGCGAGATCAAGCAGA-GGCTGAAGC-TGAAGGACGGCGGCCACTACGACGCTGAGGTCAAGACCACCTACA-AGGCCAAGAAG-CCCGTGCAGCTGCCCGGC-GCCTACAACGTCAACATCAAGT-TG----GA-CATCACCTCCCACAACGAGGA-CTAC-A-C-CA---T-C-G-TGGAACAGTACG-AACGCGCCGAGGGCCGCCACTCCAC-CGGCGGCATGGACGAGCTGTACAAG</th></tr><tr><th>y:</th><th>ATGGTGAGCAAGGGCGAGGA-G----C-T-G--TTCA-C-CGG-GGTGGTGCCCATCCTGGT-CGAGC-TGGACGGCGACGTAAACGGCCACAAGTTC-AG--CGTGTCCGGCGAGGGCGAGGGCGATGCCACCTAC---GGCAAGCTGACC-CTGAAG-TTCATTTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCC-AC-CCTCGTGACCACCCTGACCTACGGCGTGCAGTGC-T-TCAGCCGCTACCCCGACC-ACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGC-GCACCATCTTCTTCAAGGACGACGGCAACTACAAGA-CCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGC-A--ACATC--C-TGGGGCACAAGCTG-G-AGTA-CAACTACAACAGCC-ACAACGTC-TATAT-CATG--GCCGA-CAA--GCAGAAGAACGG-CA--T-C-A-AGG-TGAACTTC-AAGATC--CGCCAC--AA---C---ATCGAG--GACGGC---AGCGTGCAGCTCGCCGACCACTACCA-GC--A-G--AACACC-CC--CATCGGCGACG--GCCCCGTGCTGCTGCCCGACAACC-ACTACCTGAGCACCCAGTCCGCCCTGAGCAA-A-GACCC-CAACGAGAAGC-GCGATCACATGGTCCTGCTGG---AGTTCGTGAC-CGCC----GCCGGGA-T-CACTC-TCGGCATGGACGAGCTGTACAAG</th></tr></table>")

x:,ATGGTGAGCAAGGGCGAGGAGGATAACATGGCCATCATCAAGGAGTTCATGCGCTTCAAGGTGC-A-CATGGAGGGCTCCGTGAACGGCCACGAGTTCGAGATCGAG---GGCGAGGGCGAGGGC--CGCC-CCTACGAGGGCACCCAGACCGC-CAAGCTGAAGGTG-ACCA-AGG---G-TGGCC---CCCT-GCCCTTCGCCT-GGGA-CATCCTGTCC--C--C-T-CAGTTCATGT-A-CGGCT-CCAAGGCCTACGTG-A--AGCAC--C--C--C--G-CCGACATCCCCG-A--CTAC-T--TGAAGCTG-TCCTTC--C--C-----CGA-GG--GCTTCAAGTGGGAGCG-CGTGATGAACTTCGAGGACGGCGGCGTGGTG-ACCG--T-GA-C-CCAGGAC-TC--CTCCCTGCAGGACGGCGAGTTCATCTACAAGGTG---AAGCTGCGCGGCACCAACTTCCCCT-CCGACGGCCCCGTA-ATGCA-GAAGAAGACCATGGGCTG--GGA-GGCCTCCTCCGAGCGGATGTACCCCGAGGA-CGGCGCC-CTGAAGGGCGAGATCAAGCAGA-GGCTGAAGC-TGAAGGACGGCGGCCACTACGACGCTGAGGTCAAGACCACCTACA-AGGCCAAGAAG-CCCGTGCAGCTGCCCGGC-GCCTACAACGTCAACATCAAGT-TG----GA-CATCACCTCCCACAACGAGGA-CTAC-A-C-CA---T-C-G-TGGAACAGTACG-AACGCGCCGAGGGCCGCCACTCCAC-CGGCGGCATGGACGAGCTGTACAAG
y:,ATGGTGAGCAAGGGCGAGGA-G----C-T-G--TTCA-C-CGG-GGTGGTGCCCATCCTGGT-CGAGC-TGGACGGCGACGTAAACGGCCACAAGTTC-AG--CGTGTCCGGCGAGGGCGAGGGCGATGCCACCTAC---GGCAAGCTGACC-CTGAAG-TTCATTTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCC-AC-CCTCGTGACCACCCTGACCTACGGCGTGCAGTGC-T-TCAGCCGCTACCCCGACC-ACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGC-GCACCATCTTCTTCAAGGACGACGGCAACTACAAGA-CCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGC-A--ACATC--C-TGGGGCACAAGCTG-G-AGTA-CAACTACAACAGCC-ACAACGTC-TATAT-CATG--GCCGA-CAA--GCAGAAGAACGG-CA--T-C-A-AGG-TGAACTTC-AAGATC--CGCCAC--AA---C---ATCGAG--GACGGC---AGCGTGCAGCTCGCCGACCACTACCA-GC--A-G--AACACC-CC--CATCGGCGACG--GCCCCGTGCTGCTGCCCGACAACC-ACTACCTGAGCACCCAGTCCGCCCTGAGCAA-A-GACCC-CAACGAGAAGC-GCGATCACATGGTCCTGCTGG---AGTTCGTGAC-CGCC----GCCGGGA-T-CACTC-TCGGCATGGACGAGCTGTACAAG


In [55]:
genbank1 = ("atggtgagcaagggcgaggaggataacatggccatcatcaaggagttcatgcgcttcaaggtgcacatggagggctccgtgaacggccacgagttcgagatcgagggcgagggcgagggccgcccctacgagggcacccagaccgccaagctgaaggtgaccaagggtggccccctgcccttcgcctgggacatcctgtcccctcagttcatgtacggctccaaggcctacgtgaagcaccccgccgacatccccgactacttgaagctgtccttccccgagggcttcaagtgggagcgcgtgatgaacttcgaggacggcggcgtggtgaccgtgacccaggactcctccctgcaggacggcgagttcatctacaaggtgaagctgcgcggcaccaacttcccctccgacggccccgtaatgcagaagaagaccatgggctgggaggcctcctccgagcggatgtaccccgaggacggcgccctgaagggcgagatcaagcagaggctgaagctgaaggacggcggccactacgacgctgaggtcaagaccacctacaaggccaagaagcccgtgcagctgcccggcgcctacaacgtcaacatcaagttggacatcacctcccacaacgaggactacaccatcgtggaacagtacgaacgcgccgagggccgccactccaccggcggcatggacgagctgtacaagtaa").upper()
genbank2 = ("atggtctccttcaaatctctcctagttctctgttgcgctgcccttggggcattcgctacgaagagaatgagtaaaggagaagaacttttcactggagttgtcccaattcttgttgaattagatggtgatgttaatgggcacaaattttctgtcagtggagagggtgaaggtgatgcaacatacggaaaacttacccttaaatttatttgcactactggaaaactacctgttccatggccaacacttgtcactactttcacctatggtgttcaatgcttttcaagatacccagatcatatgaagcggcacgacttcttcaagagcgccatgcctgagggatacgtgcaggagaggaccatcttcttcaaagacgacgggaactacaagacacgtgctgaagtcaagtttgagggagacaccctcgtcaacaggatcgagcttaagggaatcgatttcaaggaggacggaaacatcctcggccacaagttggaatacaactacaactcccacaacgtatacatcatggccgacaagcaaaagaacggcatcaaagccaacttcaagacccgccacaacatcgaagacggcggcgtgcaactcgctgatcattatcaacaaaatactccaattggcgatggccctgtccttttaccagacaaccattacctgtccacacaatctgccctttcgaaagatcccaacgaaaagagagaccacatggtccttcttgagtttgtaacagctgctgggattacacatggcatggatgaactatacaaataa").upper()

S,B = sw_fwd(genbank1, genbank2, cmap, m, go, ge)

print(sw_bwd(genbank1, genbank2, S, B))

('TGGTGAGCA__AG_GGCG_AG_G_AGGA_TAACATG_G__CCA_TCATCA_AGGAGTTCATGCGC__TTCAAGGTGCACA_TGGA_GG_GCTCCGTGAACGGCCAC_GA_GTTCGAGATCGAG_GGCGAGGGCG_AGG_GCCGC_CCCTACGAGGGCACCCAGACCGC_CAAGCTGAAGGTGAC_CAAGGGTGGCCCCCTGCC_CTTCGCCTGG__GACATCCTGTCCCCTCA_GTTCATGTACGG_CTCCAAGGCCTACGTGAAG_CACCCCG__C___CGACATC__CCCGACTACTT_GA_AGCTGTCCTTCCCCGAGGGCTTCAAGTG__GGAGCGCG_TGATGAACTTC___GAGGAC_GG_____C__G__GCGTGGTG_A__C_CG__TGACCCAGGAC_TCCTC_CCTGCAGGA_CG_GC____GAG_TTC_ATCTACAAGGTGAAGCTGCGCGGCACCAACTTCCCCTCCGACGGCCCCGTAATGCAGAAGAAGACCATGGGCTGGGAGGC__CTCCTC_CGAGCGGATGTACCCCGA_GGACGGC__GCCCTGAAGGGCGAG_ATC_AAG_C_A____GAG_GCTG__A_AGC_T_GAAGGACGGCGGC___C_ACTACGAC_G__C_TGAGGTCAAGACCA_CCTACAA__GGCCAAGAAGCCC_GTGCAGCTGCCCGGC_GCC__TACAAC_GTCAACATCAAGT_TGGAC_ATC___ACCTCCC_AC__AACGAG_GACTACA____CCATCGTGGAACAGTACG_AAC_GC_GCCGAGGGCCGCCACTCCACCGGCGGCATGGACGAGCTGTACAAGTAA', 'TGG_G_GCATTCGCTACGAAGAGAATGAGTAA_AGGAGAAGAACTTTTCACTGGAGTT_GT_CCCAATTCTTGTTG_A_ATTAGATGGTGAT__GTTAATGGGCACAAATTTTC_TG_TC_AGTGGAGAGGGTGAAGGTGATGCAACATAC___GGAAAA

# Exercice 3 : Distribution des scores d’alignement pour des séquences aléatoires

Pour tester si un alignement reflète une réelle similarité biologique, on va évaluer la distribution des scores d’alignement pour des paires de séquences aléatoires.

Q1. En considérant deux séquences aléatoires de même taille N, où chaque nucléotide apparaît avec une probabilité uniforme de ¼, calculer le score moyen attendu pour une superposition sans trou dans le cas où une identité vaut +1 et une différence vaut 0.

```markdown
Votre réponse ici
```

Q2. La question précédente peut se resoudre analytiquement car on ne considère pas de trou. Pour étendre le résultat precedent à un alignement avec trous, on va se baser sur la simulation de séquences aleatoires.

Générez $R$ paires de séquences aléatoires  de tailles $N$ avec des probabilitées uniformes d'apparition de nucléotides $p_A = p_T = p_G = p_C = $ ¼. Affichez sous forme de violinplots les distribution des scores d'alignements entre chaque paire, obtenu par :
  1. un alignement sans trou (cf. Q1) ;
  2. un alignement local via Smith-Waterman (utilisez le code de l'exercice précédent)

Utilisez le schéma d'évaluation suivant :

In [ ]:
rmap = {"A": 0, "T": 1, "G": 2, "C": 3}
sigma = np.array([[1, -0.5, -0.5, -0.5],
                  [-0.5, 1, -0.5, -0.5],
                  [-0.5, -0.5, 1, -0.5],
                  [-0.5, -0.5, -0.5, 1]])
go =0
ge = 0.5

In [ ]:
#Votre code ici

Q3. Qu'observez-vous ?

```markdown
Votre réponse ici
```

Q4. Quelle conclusion peut-on en tirer sur la significativité d'un alignement ?

```markdown
Votre réponse ici
```